In [17]:
import json
from pathlib import Path
import numpy as np
from PIL import Image
from collections import defaultdict

In [2]:
data_path = Path("/mnt/vol_c/data/3d-future/")

In [3]:
meta = json.loads((data_path / "meta_data/split/train_all.json").read_text())

In [4]:
threedfuture = list(filter(lambda d: d['dataset'] == '3D-FUTURE', meta))
len(threedfuture)

16563

In [5]:
for d in threedfuture:
    d['data_path'] = d['data_path'].replace('/mnt/data/', '/mnt/vol_c/data/3d-future/')

In [6]:
all_objects = []
for m in threedfuture:
    dict = np.load(m['data_path'], allow_pickle=True)[()]
    all_objects.append(dict)

len(all_objects)

16563

In [7]:
type = "armchair"
views_per_object = 4 # out of 12 possible
file_name = "armchairs"

In [8]:
objects = list(filter(lambda x: x['text'][0] == type, all_objects))
len(objects)

751

In [11]:
objects[0]

{'dataset': '3D-FUTURE',
 'group': '',
 'id': '5554e6fe-082d-31ce-a307-0394e3d80206',
 'text': ['armchair', 'armchair, Southeast Asia, Smooth Net, Leather'],
 'text_feat': [{'original': array([[ 0.30552682,  0.23419845,  0.6894305 , ..., -1.4306782 ,
           -0.3754994 , -1.2304761 ]], dtype=float32),
   'prompt_avg': array([[-0.2673788 ,  0.3053051 , -0.1786079 , ..., -0.7090513 ,
           -2.2703962 , -0.17871311]], dtype=float32)},
  {'original': array([[ 1.4308784 , -0.1345261 , -0.74500465, ..., -0.88641554,
           -1.4061755 , -0.94789904]], dtype=float32),
   'prompt_avg': array([[ 1.1871456 ,  0.06281681, -0.58862805, ..., -0.86801815,
           -1.3048692 , -0.6447348 ]], dtype=float32)}],
 'blip_caption': 'a gray couch on a white background',
 'blip_caption_feat': {'original': array([[ 0.42915505,  0.9947657 , -0.8185683 , ..., -0.95932794,
          -0.94799805,  0.32492948]], dtype=float32),
  'prompt_avg': array([[ 0.07374929,  0.7686799 , -0.5967371 , ..., -0.70

In [12]:
import random

shuffled = random.shuffle(objects)

split_idx = int(0.75*len(objects))
train = objects[:split_idx]
test = objects[split_idx:]

In [13]:
len(train)

563

In [14]:
len(test)

188

In [21]:
def sample_views(objects: list[dict]) -> list[dict]:
    data = []
    freq = defaultdict(int)
    for obj in objects:
        id = obj['id']
        text = obj['text'][1]
        freq[text.count(',')] += 1
        pointcloud = obj['xyz']
        if not (data_path / id).exists():
            continue
        if text.count(',') != 3:
            continue
        views = random.choices(list(range(8)), k=views_per_object)
        for view in views:
            image = np.array(Image.open(data_path / id / f"colors_{view}.png"))
            if np.count_nonzero(image[:, :, 3]) == 0:
                print(id)
                continue
            data.append({
                'id': id,
                'text': text,
                'view': view,
                'pointcloud': pointcloud
            })
    print(freq)
    random.shuffle(data)
    return data

In [22]:
train_final = sample_views(train)
len(train_final)

5eabe872-cabc-3c9a-8a0e-0fc3c375d60c
5eabe872-cabc-3c9a-8a0e-0fc3c375d60c
5eabe872-cabc-3c9a-8a0e-0fc3c375d60c
5eabe872-cabc-3c9a-8a0e-0fc3c375d60c
195d7577-46eb-331d-afb2-5213917aa50d
195d7577-46eb-331d-afb2-5213917aa50d
195d7577-46eb-331d-afb2-5213917aa50d
195d7577-46eb-331d-afb2-5213917aa50d
7ff61f65-2460-37db-bb55-4da380a92de1
7ff61f65-2460-37db-bb55-4da380a92de1
7ff61f65-2460-37db-bb55-4da380a92de1
7ff61f65-2460-37db-bb55-4da380a92de1
85088488-cf38-3193-ad59-0363a33681c3
85088488-cf38-3193-ad59-0363a33681c3
85088488-cf38-3193-ad59-0363a33681c3
85088488-cf38-3193-ad59-0363a33681c3
7eb8707b-2853-3455-9c6f-09313fc2b4e0
7eb8707b-2853-3455-9c6f-09313fc2b4e0
7eb8707b-2853-3455-9c6f-09313fc2b4e0
7eb8707b-2853-3455-9c6f-09313fc2b4e0
6494f33c-0c80-3b87-934f-2645b515f81b
6494f33c-0c80-3b87-934f-2645b515f81b
6494f33c-0c80-3b87-934f-2645b515f81b
6494f33c-0c80-3b87-934f-2645b515f81b
4159b181-95ea-3f6a-b68b-8131e75c755e
4159b181-95ea-3f6a-b68b-8131e75c755e
4159b181-95ea-3f6a-b68b-8131e75c755e
4

2192

In [23]:
test_final = sample_views(test)
len(test_final)

4ecfdba8-5420-343d-af07-cbf6df2facc6
4ecfdba8-5420-343d-af07-cbf6df2facc6
4ecfdba8-5420-343d-af07-cbf6df2facc6
4ecfdba8-5420-343d-af07-cbf6df2facc6
4b9c54b8-2b9f-3da6-8395-b13dbe268eee
4b9c54b8-2b9f-3da6-8395-b13dbe268eee
4b9c54b8-2b9f-3da6-8395-b13dbe268eee
4b9c54b8-2b9f-3da6-8395-b13dbe268eee
defaultdict(<class 'int'>, {3: 185, 1: 3})


728

In [24]:
dataset = {
    'train': train_final,
    'test': test_final
}

In [25]:
import pickle

with open(data_path / (file_name + '.pkl'), 'wb') as f:
    pickle.dump(dataset, f)